In [0]:
from pyspark.sql import Window
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
forex_df = spark.read.format("delta").load("/mnt/forexratesdl/processed/forex_data")
forex_df.createOrReplaceTempView("tempviewforex")

In [0]:
forex_parkSQL = spark.sql("WITH Cte AS (SELECT base_currency_id,target_currency_id,currency_name,amount,data_time,\
    LAG(amount) OVER(PARTITION BY base_currency_id,target_currency_id ORDER BY data_time) AS previus_amount,\
    ABS(amount - LAG(amount) OVER (PARTITION BY base_currency_id, target_currency_id ORDER BY data_time)) AS daily_change\
    FROM tempviewforex)\
    SELECT base_currency_id,target_currency_id,currency_name,round(daily_change,3) as daily_change ,data_time\
    FROM Cte ORDER BY daily_change DESC")
forex_parkSQL = forex_parkSQL.where(to_date('data_time', 'yyyy/MM/dd HH:mm:ss.SSS') == current_date())

base_currency_id,target_currency_id,currency_name,daily_change,data_time
USD,PYG,Paraguayan Guarani,15.213,2024-04-29T14:00:00Z
USD,COP,Colombian Peso,10.48,2024-04-29T14:00:00Z
USD,CRC,Costa Rican Colon,7.084,2024-04-29T14:00:00Z
USD,CLP,Chilean Peso,4.481,2024-04-29T14:00:00Z
USD,ARS,Argentine Peso,2.926,2024-04-29T14:00:00Z
USD,DOP,Dominican Peso,0.333,2024-04-29T14:00:00Z
USD,UYU,Uruguayan Peso,0.274,2024-04-29T14:00:00Z
USD,NIO,Nicaraguan Cordoba,0.176,2024-04-29T14:00:00Z
USD,VES,Venezuelan Bolovar Soberano,0.065,2024-04-29T14:00:00Z
USD,PEN,Peruvian Nuevo Sol,0.039,2024-04-29T14:00:00Z


In [0]:
forex_parkSQL.write.mode("append").format("delta").saveAsTable("presentation_forex_db.daily_change")